# Set Up
If you haven't setup your credentials in the config.json file, please do so in the given prompt.

In [9]:
import pandas as pd
import requests
import json

In [10]:
# Setting up the config for usage.
try:
    with open('config.json', 'r') as f:
        config = json.load(f)
    user_agent = config['user_agent']
    key1 = config['key1']
    key2 = config['key2']
    key3 = config['key3']
    username = config['username']
    print("Config Loaded Successfully.")
    key_list = [key1, key2, key3]
    
except:
    print('config.json wasn\'t found.\nCreating new config file. Please enter your credentials.')
    user_agent = input('Enter User Agent: ')
    username = input('Enter your username: ')
    key1 = input('Enter Secret Key: ')
    key2 = input('Enter Secret Key 2:\nLeave blank if you don\'t have mirror keys. ')
    if key2 == '':
        key2 = key1
    key3 = input('Enter Secret Key 3:\nLeave blank if you don\'t have mirror keys. ')
    if key3 == '':
        key3 = key1
    
#     with open('config.json.sample','r') as f:
#         config = json.load(f)
    config = {'user_agent':None,'key1':None, 'key2':None, 'key3':None, 'username':None}
    config['user_agent'] = user_agent
    config['key1'] = key1
    config['key2'] = key2
    config['key3'] = key3
    config['username'] = username
    
    with open('config.json','w+') as f:
        json.dump(config, f)
    
    key_list = [key1, key2, key3]
     
    print()
    print("Your configuration has been saved to config.json!")
    
# Fetches variables: user_agent, key1, key2, key3, key_list

Config Loaded Successfully.


# Fetching Data from Last.fm

In [11]:
# Generic Function for sending requests

def send_request(user=username, page=1, limit=200, key=key1, method='user.getrecenttracks'):
    
    headers = {'user-agent': user_agent}
    
    payload ={
    'limit':limit,
    'user':user,
    'page': page,
    'api_key': key,
    'method': method,
    'format': 'json'
    }
    
    api_reply = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
    
    return api_reply

In [17]:
# Fetch a page and clean data before return.
# Also handles exceptions.

def get_page(num=1, username=username, key=key1, method='user.getrecenttracks'):
    
    try:
        response = send_request(page=num, key=key, method=method)
        
        if response.status_code != 200:
            raise Exception(api_reply.text)
        
        page = (response.json())['recenttracks']['track']
        
        if num == 1: 
            global totalPages
            totalPages = int((response.json())['recenttracks']['@attr']['totalPages'])
        
        if (num!=1) & ('@attr' in page[0].keys()): 
            if (page[0]['@attr']=={'nowplaying': 'true'}):
                del page[0]
        
        return page

    except:
        raise Exception("request status {}".format(response.status_code))

# Get the total number of pages for 200 scrobbles per page.

def get_totalPages(method='user.getrecenttracks'):
    response = send_request(method = method)
    totalPages = int((response.json())['recenttracks']['@attr']['totalPages'])
    return totalPages

totalPages = get_totalPages()

In [18]:
def key_picker(num, ls):
    index_num = num % len(ls)
    return ls[index_num]

In [ ]:
from IPython.core.display import clear_output

pages = []

for n in range(1, (totalPages)+1):
    
    print('fetched page {}/{}'.format(n, totalPages))
    
    spaces = (((totalPages-n)/totalPages)*50)
    bar = ((n/totalPages)*50)
    string = '[{}{}] {}% Completed'.format('~'*int(bar),' '*int(spaces), int(bar)*2)
    print(string)
    
    clear_output(wait=True)
    
    ky = key_picker(num = n, ls = key_list)
    fetched_page = get_page(num=n, key=ky)
    
    pages = [*pages, *fetched_page]
print()
print("Fetched {} pages, {} scrobbles".format(totalPages, len(pages)))

fetched page 12/62
[~~~~~~~~~                                        ] 18% Completed


# Cleaning up Fetched Data with #Pandas

In [ ]:
# Under Construction!